<a href="https://colab.research.google.com/github/khariton1/GlobeX/blob/main/BROMKHcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import hashlib
import time

class Block:
    def __init__(self, index, previous_hash, timestamp, data, difficulty, nonce, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.difficulty = difficulty
        self.nonce = nonce
        self.hash = hash

    def __repr__(self):
        return (f"Block(Index: {self.index}, Hash: {self.hash}, Previous Hash: {self.previous_hash}, "
                f"Data: {self.data}, Nonce: {self.nonce}, Difficulty: {self.difficulty})")

def calculate_hash(index, previous_hash, timestamp, data, difficulty, nonce):
    value = str(index) + str(previous_hash) + str(timestamp) + str(data) + str(difficulty) + str(nonce)
    return hashlib.sha256(value.encode()).hexdigest()

def create_genesis_block(difficulty):
    timestamp = time.time()
    return Block(0, "0", timestamp, "Genesis Block", difficulty, 0,
                 calculate_hash(0, "0", timestamp, "Genesis Block", difficulty, 0))

def mine_block(index, previous_hash, timestamp, data, difficulty):
    nonce = 0
    while True:
        hash_value = calculate_hash(index, previous_hash, timestamp, data, difficulty, nonce)
        if hash_value[:difficulty] == "0" * difficulty:
            return Block(index, previous_hash, timestamp, data, difficulty, nonce, hash_value)
        nonce += 1

def create_new_block(previous_block, data, difficulty):
    index = previous_block.index + 1
    timestamp = time.time()
    previous_hash = previous_block.hash
    return mine_block(index, previous_hash, timestamp, data, difficulty)
class Wallet:
    def __init__(self, brh_balance=1000, usdt_balance=100):
        self.brh_balance = brh_balance
        self.usdt_balance = usdt_balance

    def __repr__(self):
        return f"Wallet(BRH: {self.brh_balance}, USDT: {self.usdt_balance})"

    def debit(self, brh_amount, usdt_amount):
        if self.brh_balance >= brh_amount and self.usdt_balance >= usdt_amount:
            self.brh_balance -= brh_amount
            self.usdt_balance -= usdt_amount
            return True
        return False

    def credit(self, brh_amount, usdt_amount):
        self.brh_balance += brh_amount
        self.usdt_balance += usdt_amount

class Blockchain:
    def __init__(self, difficulty, developer_wallet, mining_fee_percentage=5):
        self.chain = [create_genesis_block(difficulty)]
        self.difficulty = difficulty
        self.developer_wallet = developer_wallet  # Кошелек разработчиков
        self.mining_fee_percentage = mining_fee_percentage  # Процент скрытой комиссии при майнинге
        self.nfts = {}  # Словарь для хранения NFT (уникальные токены)
        self.create_nft_fee = 10  # Комиссия за создание NFT в BRH
        self.copy_nft_fee = 100  # Комиссия за копирование NFT в BRH
        self.create_token_usdt_fee = 3  # Комиссия в USDT за создание токенов (до 1 млн)
        self.create_token_brh_fee = 5   # Комиссия в BRH за создание токенов (до 1 млн)
        self.token_limit = 1000000  # Порог в 1 млн токенов для фиксированной комиссии
        self.contracts = {}  # Хранилище для смарт-контрактов

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        new_block = create_new_block(latest_block, data, self.difficulty)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != calculate_hash(current_block.index, current_block.previous_hash,
                                                   current_block.timestamp, current_block.data,
                                                   current_block.difficulty, current_block.nonce):
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True

    def mine_tokens(self, wallet, amount):
        # Рассчитываем скрытую комиссию
        mining_fee = amount * self.mining_fee_percentage / 100
        user_tokens = amount - mining_fee

        # Кошелек майнера получает остаток токенов
        wallet.credit(user_tokens, 0)

        # Кошелек разработчиков получает комиссию
        self.developer_wallet.credit(mining_fee, 0)

        return user_tokens, mining_fee
    def create_nft(self, wallet, nft_id, nft_data):
        # Создание уникального NFT с идентификатором и данными
        if nft_id not in self.nfts:
            if wallet.debit(self.create_nft_fee, 0):  # Списание BRH за создание NFT
                self.nfts[nft_id] = nft_data
                return f"NFT с ID {nft_id} создан! Комиссия за создание: {self.create_nft_fee} BRH."
            return "Недостаточно средств в кошельке для создания NFT."
        return f"NFT с ID {nft_id} уже существует!"

    def copy_nft(self, wallet, nft_id, new_nft_id):
        # Копирование существующего NFT в новый NFT
        if nft_id in self.nfts:
            if new_nft_id not in self.nfts:
                if wallet.debit(self.copy_nft_fee, 0):  # Списание BRH за копирование NFT
                    self.nfts[new_nft_id] = self.nfts[nft_id]
                    return (f"NFT с ID {nft_id} скопирован в новый NFT с ID {new_nft_id}! "
                            f"Комиссия за копирование: {self.copy_nft_fee} BRH.")
                return "Недостаточно средств в кошельке для копирования NFT."
            return f"NFT с ID {new_nft_id} уже существует!"
        return f"NFT с ID {nft_id} не найден."

    def create_tokens(self, wallet, amount):
        # Создание новых токенов с фиксированной комиссией для объёма до 1 миллиона токенов
        if amount <= self.token_limit:
            if wallet.debit(self.create_token_brh_fee, self.create_token_usdt_fee):  # Списание комиссии в BRH и USDT
                return (f"Создано {amount} токенов! Комиссия за создание: {self.create_token_usdt_fee} USDT "
                        f"и {self.create_token_brh_fee} BRH.")
            return "Недостаточно средств в кошельке для создания токенов."
        else:
            return "Комиссия для создания более 1 миллиона токенов не установлена."
class SmartContract:
    def __init__(self, owner_wallet, contract_code, initial_data):
        self.owner_wallet = owner_wallet
        self.contract_code = contract_code  # Код контракта, который будет исполняться
        self.state = initial_data  # Начальное состояние контракта

    def execute(self, blockchain, params):
        # Исполнение контракта с передачей параметров и обновлением состояния
        exec(self.contract_code, {"blockchain": blockchain, "params": params, "state": self.state})

class BlockchainWithContracts(Blockchain):
    def deploy_contract(self, wallet, contract_code, initial_data):
        # Разворачивание контракта в сети
        contract_id = len(self.contracts) + 1
        contract = SmartContract(wallet, contract_code, initial_data)
        self.contracts[contract_id] = contract
        return f"Смарт-контракт {contract_id} развёрнут!"

    def execute_contract(self, contract_id, params):
        # Исполнение контракта по его ID
        if contract_id in self.contracts:
            contract = self.contracts[contract_id]
            contract.execute(self, params)
            return f"Контракт {contract_id} исполнен!"
        return "Контракт не найден!"
# Пример кода смарт-контракта, который переводит BRH с одного кошелька на другой
contract_code_transfer = """
if params['from_wallet'].debit(params['amount'], 0):
    params['to_wallet'].credit(params['amount'], 0)
    state['last_transaction'] = params['amount']
"""

# Создаем кошельки
wallet_1 = Wallet()
wallet_2 = Wallet()
developer_wallet = Wallet()

# Создаем блокчейн с контрактами
brh21_blockchain = BlockchainWithContracts(4, developer_wallet)

# Разворачиваем контракт перевода средств
contract_deploy_result = brh21_blockchain.deploy_contract(wallet_1, contract_code_transfer, {"last_transaction": 0})
print(contract_deploy_result)

# Исполняем контракт перевода 100 BRH с кошелька 1 на кошелек 2
contract_execution_result = brh21_blockchain.execute_contract(1, {"from_wallet": wallet_1, "to_wallet": wallet_2, "amount": 100})
print(contract_execution_result)
print(f"Баланс кошелька 1: {wallet_1}")
print(f"Баланс кошелька 2: {wallet_2}")
# Майним несколько блоков
brh21_blockchain.add_block("Первый блок в сети BRH 21")
brh21_blockchain.add_block("Второй блок в сети BRH 21")

# Выводим всю цепочку блоков сети BRH 21
print("\nЦепочка блоков сети BRH 21:")
for block in brh21_blockchain.chain:
    print(block)

# Проверяем, является ли цепочка валидной
print("\nЦепочка BRH 21 валидна:", brh21_blockchain.is_chain_valid())

# Майним токены на кошелек 2
mined_tokens_2, mining_fee_2 = brh21_blockchain.mine_tokens(wallet_2, 100)
print(f"\nВы замайнили {mined_tokens_2} токенов в сети BRH 21. Комиссия разработчикам: {mining_fee_2} BRH.")
print(f"Баланс кошелька 2: {wallet_2}")
print(f"Баланс кошелька разработчиков после майнинга: {developer_wallet}")

# Создание NFT в сети BRH 21 с кошелька 2
nft_create_response_2 = brh21_blockchain.create_nft(wallet_2, "NFT2", "Уникальные данные NFT2")
print(f"\n{nft_create_response_2}")
print(f"Баланс кошелька 2 после создания NFT: {wallet_2}")

# Копирование NFT в сети BRH 21 с кошелька 2
nft_copy_response = brh21_blockchain.copy_nft(wallet_2, "NFT2", "NFT3")
print(f"\n{nft_copy_response}")
print(f"Баланс кошелька 2 после копирования NFT: {wallet_2}")
!pip install Flask
# ... Код блокчейна и других классов ...

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/mine', methods=['POST'])
def mine_block():
    data = request.json['data']
    brh21_blockchain.add_block(data)
    return jsonify({"message": "Block mined successfully!"})

@app.route('/chain', methods=['GET'])
def get_chain():
    chain_data = []
    for block in brh21_blockchain.chain:
        chain_data.append({
            "index": block.index,
            "previous_hash": block.previous_hash,
            "timestamp": block.timestamp,
            "data": block.data,
            "hash": block.hash,
            "nonce": block.nonce
        })
    return jsonify(chain_data)

@app.route('/validate', methods=['GET'])
def validate_chain():
    is_valid = brh21_blockchain.is_chain_valid()
    return jsonify({"message": "Chain is valid" if is_valid else "Chain is not valid"})

@app.route('/create_nft', methods=['POST'])
def create_nft():
    wallet_id = request.json['wallet_id']
    nft_id = request.json['nft_id']
    nft_data = request.json['nft_data']
    wallet = get_wallet(wallet_id)  # Необходимо определить функцию get_wallet()

    response = brh21_blockchain.create_nft(wallet, nft_id, nft_data)
    return jsonify({"message": response})

# Запуск Flask-сервера
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Смарт-контракт 1 развёрнут!
Контракт 1 исполнен!
Баланс кошелька 1: Wallet(BRH: 900, USDT: 100)
Баланс кошелька 2: Wallet(BRH: 1100, USDT: 100)

Цепочка блоков сети BRH 21:
Block(Index: 0, Hash: fed70ed7a8b1e34cb5bcee429d5c22111be064104818d249967d120b464f11e5, Previous Hash: 0, Data: Genesis Block, Nonce: 0, Difficulty: 4)
Block(Index: 1, Hash: 0000d864b0529b0e81ccbd61dc431cc6a5523ed2a5c1868a1f76c4256e1f93b7, Previous Hash: fed70ed7a8b1e34cb5bcee429d5c22111be064104818d249967d120b464f11e5, Data: Первый блок в сети BRH 21, Nonce: 8492, Difficulty: 4)
Block(Index: 2, Hash: 0000129f27533c29c92182a6f7a95cd5fc6d9128a28732342f12de795fea0a6d, Previous Hash: 0000d864b0529b0e81ccbd61dc431cc6a5523ed2a5c1868a1f76c4256e1f93b7, Data: Второй блок в сети BRH 21, Nonce: 2844, Difficulty: 4)

Цепочка BRH 21 валидна: True

Вы замайнили 95.0 токенов в сети BRH 21. Комиссия разработчикам: 5.0 BRH.
Баланс кошелька 2: Wallet(BRH: 1195.0, USDT: 100)
Баланс кошелька разработчиков после майнинга: Wallet(BRH: 10

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
